# NLP4Stat - Query results from the Knowledge DB Demo

11-06-2021

Content of the notebook : 
- connection to the DB
- get the SQL tables'names
- load some of them from the content, get a title 
- KDB already populated (last demo), so just get some content from that
- make a small query builder (screenshots 3.1), for glossary and articles 

## Library import and connections to the DB

In [9]:
import os 
import re
import logging
import sys
import pyodbc
import hashlib
import pandas as pd
import numpy as np
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST, GET
from SPARQLWrapper import JSON, INSERT, DELETE
import sparql_dataframe

In [10]:
def connect_db(DSN, DBA, UID, PWD):

    connection = pyodbc.connect('DSN={};DBA={};UID={};PWD={}'.format(DSN, 
                                                                     DBA,
                                                                     UID,
                                                                     PWD))
    cursor = connection.cursor()

    return connection, cursor


def connect_virtuoso(DSN, UID, PWD):

    sparql = SPARQLWrapper(DSN)
    sparql.setHTTPAuth(DIGEST)
    sparql.setCredentials(UID, PWD)
    sparql.setMethod(GET)

    return sparql


In [11]:
# Connection to CDB 
connection, cursor = connect_db('Virtuoso All', 
                                'ESTAT', 
                                'LOGIN', 
                                'PASSWORD')


# Connection to the KDB 
endpoint = "http://virtuoso-test.kapcode.fr:8890/sparql/"
sparql = connect_virtuoso(endpoint, 
                          'LOGIN', 
                          'PASSWORD')


## Function definition for query to the Content DB, tables querying

In [12]:
def select_query(columns, table, conditions=None): 

    if conditions:

        query = """

            SELECT {}
            FROM {}
            WHERE {}

        """.format(columns, table, conditions)

    else:

        query = """
        SELECT {}
        FROM {}
        """.format(columns, table)

    return query


#### Get all tables name from the CDB 

In [13]:
ESTAT_V1_tables_names = pd.read_sql(select_query('*',
                                          'ESTAT.information_schema.tables', 
                                          ''), 
                             connection)
ESTAT_V1_tables_names['TABLE_NAME']

0                  dat_3_grams
1                  dat_4_grams
2          dat_4_grams_pickled
3                  dat_article
4        dat_article_paragraph
5      dat_article_shared_link
6             dat_further_info
7                 dat_glossary
8                dat_link_info
9                  dat_link_tm
10        dat_paragraph_figure
11            dat_redirections
12        dat_related_concepts
13                 dat_sources
14        dat_statistical_data
15              dat_tm_results
16        dat_tm_results_words
17        mod_article_division
18    mod_resource_information
19           mod_resource_type
20            mod_topic_models
Name: TABLE_NAME, dtype: object

#### Get tables from the CDB 

In [14]:
#Load dat_link_info 
ESTAT_V1_dat_link_info = pd.read_sql(select_query('*',
                                          'ESTAT.V1.dat_link_info', 
                                          ''), 
                             connection)
print(ESTAT_V1_dat_link_info.shape)
#Load dat_glossary
ESTAT_V1_dat_glossary = pd.read_sql(select_query('*',
                                          'ESTAT.V1.dat_glossary', 
                                          ''), 
                             connection)

print(ESTAT_V1_dat_glossary.shape)
#Load dat_articles 
ESTAT_V1_dat_articles = pd.read_sql(select_query('*',
                                          'ESTAT.V1.dat_article', 
                                          ''), 
                             connection)
print(ESTAT_V1_dat_articles.shape)


(6373, 5)
(1311, 6)
(506, 6)


In [15]:
ESTAT_V1_dat_link_info.head()

,id,title,url,resource_information_id,resource_type_id
0,1,Accidents to persons caused by rolling...,https://ec.europa.eu/eurostat/statistics-expla...,1,None
1,2,Railway,https://ec.europa.eu/eurostat/statistics-expla...,1,None
2,3,Railway safety statistics in the EU,https://ec.europa.eu/eurostat/statistics-expla...,1,None
3,4,Railway freight transport statistics,https://ec.europa.eu/eurostat/statistics-expla...,1,None
4,5,Railway passenger transport statistics - quart...,https://ec.europa.eu/eurostat/statistics-expla...,1,None


In [16]:
#Load another table : 
estat_table = pd.read_sql(select_query('*',
                                          'ESTAT.V1.dat_XXXX', 
                                          ''), 
                             connection)
print(estat_table.shape)
estat_table.head()

DatabaseError: Execution failed on sql '
        SELECT *
        FROM ESTAT.V1.dat_XXXX
        ': ('42S02', '[42S02] [OpenLink][Virtuoso ODBC Driver][Virtuoso Server]SQ074: Line 4: No table ESTAT.V1.dat_XXXX in * reference (-1) (SQLExecDirectW)')

In [17]:
ESTAT_V1_dat_glossary.head()

,id,definition,redirection,original_title,homepage,last_update
0,1,Accidents to one or more persons that are ei...,0,None,0,2021-02-03 15:06:00
1,2,A railway is a line of communication ...,0,None,0,2013-02-28 10:16:00
2,6,An accident at work in the framework ...,0,None,0,2021-03-10 11:13:00
3,10,A fatal accident at work refers to an...,0,None,0,2019-03-06 12:08:00
4,11,A non-fatal accident at work is...,0,None,0,2018-09-12 15:42:00


## Querying the populated KDB 

#### Get all descriptions from the KDB 

In [147]:
SelectMatchedElements = """
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT * FROM <https://ec.europa.eu/eurostat/resource/ontology/>
WHERE {
 ?s dct:description ?o 
}
"""

elements_df = sparql_dataframe.get(endpoint, SelectMatchedElements)
print(elements_df.shape)
elements_df.tail()

(1309, 2)


,s,o
1304,https://ec.europa.eu/eurostat/resource/ontolog...,An airport is a defined area of land or...
1305,https://ec.europa.eu/eurostat/resource/ontolog...,Adult education is specifically targeted at in...
1306,https://ec.europa.eu/eurostat/resource/ontolog...,"Actual individual consumption , abbreviated..."
1307,https://ec.europa.eu/eurostat/resource/ontolog...,"<Brief user-oriented definition, one or a few ..."
1308,https://ec.europa.eu/eurostat/resource/ontolog...,Activity rate is the percentage of acti...


In [149]:
elements_df.iloc[1306,0]

'https://ec.europa.eu/eurostat/resource/ontology/2311'

In [150]:
elements_df.iloc[1306,1]

'Actual individual consumption    , abbreviated as     AIC    , refers to all goods and services actually consumed by     households    . It encompasses consumer goods and services purchased directly by households, as well as services provided by     non-profit institutions    and the     government    for individual consumption (e.g., health and education services). In international comparisons, the term is usually preferred over the narrower concept of household consumption, because the latter is influenced by the extent to which non-profit institutions and general government act as service providers.    Although     GDP     per capita    is an important and widely used indicator of countriesâ\x80\x99 level of economic welfare, consumption per capita may be more useful for comparing the relative welfare of consumers across various countries.    AIC per capita is usually highly correlated with GDP per capita, because AIC is, in practice, by far the biggest expenditure component of GDP

## Get random glossary entry and its title : 

#### Extract a random entry of the glossary table

In [151]:
random_entry = ESTAT_V1_dat_glossary.loc[np.random.choice(np.arange(len(ESTAT_V1_dat_glossary)), 10, False),
                                   ('id','definition')].iloc[2,0]

In [152]:
random_entry

1678

In [153]:
print(ESTAT_V1_dat_link_info.loc[ESTAT_V1_dat_link_info['id'] == random_entry].iloc[0,1])
ESTAT_V1_dat_glossary.loc[ESTAT_V1_dat_glossary['id'] == random_entry].iloc[0,1]

        Income quintile group      


'  In social statistics,     income quintile groups    are computed on the basis of the total     equivalised disposable income    attributed to each member of the     household    .    The data (of each person) are ordered according to the value of the total equivalised disposable income. Four cut-point values (the so-called     quintile cut-off points    ) of income are identified, dividing the survey population into five groups equally represented by 20 % of individuals each:    first quintile group of equivalised income;     second quintile group of equivalised income;     third quintile group of equivalised income;     fourth quintile group of equivalised income;     fifth quintile group of equivalised income.    The first quintile group represents 20 % of the population with the lowest income (an income smaller or equal to the first cut-off value), and the fifth quintile group represents the 20 % of population with the highest income (an income greater than the fourth cut-off val

#### Get all elements related to this concept from the KDB : 

In [154]:

SelectStatement = """
PREFIX estat: <https://ec.europa.eu/eurostat/resource/ontology/> 
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT * FROM <https://ec.europa.eu/eurostat/resource/ontology/>
WHERE {
estat:""" + str(random_entry) +""" ?s ?o 
}

"""

statements_df = sparql_dataframe.get(endpoint, SelectStatement)
print(statements_df.shape)
statements_df.tail()


(10, 2)


,s,o
5,http://purl.org/dc/terms/type,None
6,http://www.w3.org/2004/02/skos/core#concept,1678
7,http://purl.org/dc/terms/description,"In social statistics, income quintile grou..."
8,http://www.w3.org/2004/02/skos/core#altLabel,None
9,http://www.w3.org/2004/02/skos/core#prefLabel,Income quintile group


#### Get specifically the title and the source : 

In [156]:
SelectStatement = """
PREFIX estat: <https://ec.europa.eu/eurostat/resource/ontology/> 
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT * FROM <https://ec.europa.eu/eurostat/resource/ontology/>
WHERE {
{estat:""" + str(random_entry) +""" dct:source ?o }
UNION {estat:""" + str(random_entry) +""" dct:title ?o 
}
}

"""


print(SelectStatement)
statements_df = sparql_dataframe.get(endpoint, SelectStatement)
print(statements_df.shape)
statements_df.tail()
print(statements_df.iloc[0,0])
print(statements_df.iloc[1,0])


PREFIX estat: <https://ec.europa.eu/eurostat/resource/ontology/> 
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT * FROM <https://ec.europa.eu/eurostat/resource/ontology/>
WHERE {
{estat:1678 dct:source ?o }
UNION {estat:1678 dct:title ?o 
}
}


(2, 1)
https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Income_quintile_group
Income quintile group


## Query builder back-end : string matching

Note that this matching step using the regex() function from SPARQL will be replaced by a more complex matching in the following version. This part of the notebook aims to present how matched elements are subsequently used for another query to the KDB.

In [219]:
query_element_1 = "europe"
query_element_2 = "school"

In [220]:
SelectMatchedElements = """
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?s FROM <https://ec.europa.eu/eurostat/resource/ontology/>
WHERE {
{ ?s dct:title ?o 
FILTER regex(?o, '""" + str(query_element_1) +"""', "i")
FILTER regex(?o, '""" + str(query_element_2) +"""', "i")
}
UNION { ?s dct:description ?o 
FILTER regex(?o, '""" + str(query_element_1) +"""', "i")
FILTER regex(?o, '""" + str(query_element_2) +"""', "i")
}
}
"""
print(SelectMatchedElements)
elements_df = sparql_dataframe.get(endpoint, SelectMatchedElements)
print(elements_df.shape)
elements_df.tail()



PREFIX dct: <http://purl.org/dc/terms/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?s FROM <https://ec.europa.eu/eurostat/resource/ontology/>
WHERE {
{ ?s dct:title ?o 
FILTER regex(?o, 'europe', "i")
FILTER regex(?o, 'school', "i")
}
UNION { ?s dct:description ?o 
FILTER regex(?o, 'europe', "i")
FILTER regex(?o, 'school', "i")
}
}

(5, 1)


,s
0,https://ec.europa.eu/eurostat/resource/ontolog...
1,https://ec.europa.eu/eurostat/resource/ontology/6
2,https://ec.europa.eu/eurostat/resource/ontolog...
3,https://ec.europa.eu/eurostat/resource/ontolog...
4,https://ec.europa.eu/eurostat/resource/ontolog...


In [221]:
print(elements_df.iloc[0,0])

https://ec.europa.eu/eurostat/resource/ontology/6226


#### Get title and source for each returned entry 

Every matched elements is queried by an individual SPARQL query. It is here in a simple version that will be completed to get not only the matched elements but related elements as well (datasets, articles, ...)

In [222]:
query_results_df = pd.DataFrame(columns=['title',"url"])
for i, row in elements_df.iterrows():

        element = row[0]
        element = element.replace("https://ec.europa.eu/eurostat/resource/ontology/","")
        print(element)
        SelectStatement = """
        PREFIX estat: <https://ec.europa.eu/eurostat/resource/ontology/> 
        PREFIX dct: <http://purl.org/dc/terms/>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        SELECT * FROM <https://ec.europa.eu/eurostat/resource/ontology/>
        WHERE {
        {estat:""" + str(element) +""" dct:source ?o }
        UNION {estat:""" + str(element) +""" dct:title ?o 
        }
        }

        """
        statements_df = sparql_dataframe.get(endpoint, SelectStatement)
        query_results_df = query_results_df.append({'title': statements_df.iloc[1,0],'url': statements_df.iloc[0,0]},ignore_index=True)




6226
6
783
1303
2249


In [223]:
query_results_df

,title,url
0,Key data on teaching languages at school in Eu...,http://ec.europa.eu/eurostat/product?code=EC-X...
1,Accident at work,https://ec.europa.eu/eurostat/statistics-expla...
2,Agricultural holding,https://ec.europa.eu/eurostat/statistics-expla...
3,Kitchen gardens,https://ec.europa.eu/eurostat/statistics-expla...
4,Barcelona Summit,https://ec.europa.eu/eurostat/statistics-expla...
